In [1]:
!pip install english-words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for english-words: filename=english_words-2.0.1-py3-none-any.whl size=8196238 sha256=c0cad4b2195938336313af83a2779a062749bc93fc5c98cc419ce92c9fe9c4f7
  Stored in directory: /root/.cache/pip/wheels/f0/e6/d9/16a123647999fe535f03a36e7af23eef203736d84c7ca25b0b
Successfully built english-words


In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/hate-speech-and-offensive-language-detection/train.csv',
                usecols=['tweet'])
df.head()

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
stopwords_df = pd.read_csv(filepath_or_buffer='/kaggle/input/nltk-english-stopwords/nltk_eng_stopwords.csv')
stopwords = stopwords_df['list_of_stopwords'].values.tolist() 

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
stopword_model = CountVectorizer(vocabulary=stopwords)
stopword_result = stopword_model.fit_transform(raw_documents=df['tweet'].values)
stopword_count_df = pd.DataFrame(data={'word': stopword_model.get_feature_names_out(), 'count': stopword_result.toarray().sum(axis=0)})
stopword_count_df.sort_values(ascending=False, by='count').head()

,word,count
62,the,7209
8,you,6814
86,to,5343
40,that,4043
63,and,3993


In [5]:
import numpy as np
stopword_count_df = stopword_count_df.reset_index(drop=True)
stopword_count_df['x'] = [index % 40 for index in stopword_count_df.index.tolist()]
stopword_count_df['log count'] = stopword_count_df['count'].apply(np.log)

In [6]:
from plotly.express import scatter
scatter(data_frame=stopword_count_df, x='x', y='count', text='word', log_y=True).update_traces(marker={'size': 1})

In [7]:
from plotly.express import scatter
scatter(data_frame=stopword_count_df, x='x', y='count', hover_name='word', color='log count', log_y=True)

In [8]:
from english_words import get_english_words_set
web2lowerset = get_english_words_set(['web2'], lower=True)
len(web2lowerset)

234450

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
web2_model = CountVectorizer(vocabulary=web2lowerset, stop_words=stopwords)
web2_result = web2_model.fit_transform(raw_documents=df['tweet'].values)
web2_count_df = pd.DataFrame(data={'word': web2_model.get_feature_names_out(), 
                                   'count': web2_result.toarray().sum(axis=0)})
web2_count_df.sort_values(ascending=False, by='count').head()

,word,count
22822,bitch,8348
105962,like,2787
159504,pussy,2160
85869,hoe,1951
14333,ass,1579


In [10]:
web2_count_df.shape

(234450, 2)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
residue_model = CountVectorizer(stop_words=stopwords + list(web2lowerset))
residue_result = residue_model.fit_transform(raw_documents=df['tweet'].values)
residue_count_df = pd.DataFrame(data={'word': residue_model.get_feature_names_out(), 
                                   'count': residue_result.toarray().sum(axis=0)})
residue_count_df.sort_values(ascending=False, by='count').head()

,word,count
20275,rt,7647
430,128514,3241
4107,bitches,3119
5983,co,3009
11238,http,2915
